# Cleaning and exploring data for Macro-Eyes

In [201]:
import os
import pathlib
from pathlib import Path 
import pandas as pd
import numpy as np
import re

To do:
* Remove duplicate rows
* Check for different text referring to same facility eg 'hospital' vs 'chp'

It will be helpful later to display all the columns:

In [202]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)

## Read in raw data

In [203]:
df_raw = pd.read_excel('Health Facility Assessment (1).xlsx',index_col=0, usecols='B:GW')
df = df_raw.copy()

## Delete empty columns

In [204]:
# the columns whose names end with 'choice_labels' are empty, so we can delete them
choice_label_cols = []
for col in df.columns:
    if len((re.findall('\.choice_labels',col)))!=0:
        choice_label_cols.append(col)

df = df.drop(choice_label_cols,axis=1)


## Shorten column names

Let's start by displaying the column names:

In [205]:
cols = list(df.columns)
print(*cols,sep='\n')

form.health_centre_information.facility_name
form.facility_gps
form.health_centre_information.location_information.region_province
form.health_centre_information.location_information.district
form.health_centre_information.location_information.chiefdom
form.health_centre_information.location_information.facility_location
form.health_centre_information.facility_type
form.health_centre_information.facility_type_other
form.health_centre_information.managing_authority
form.health_centre_information.managing_authority_other
form.health_centre_information.setting
form.health_centre_information.outpatient_only
form.health_centre_information.capacity.number_consultation_rooms
form.health_centre_information.capacity.number_inpatient_beds
form.health_centre_information.capacity.number_maternity_beds
form.health_centre_information.group_number_employed.list_community_health_officer.cho_number_in_post
form.health_centre_information.group_number_employed.list_community_health_officer.cho_number_pre

These column names are painfully long to work with, so I will remove some unnecesary words. 

In [206]:
new_cols = []
for i,name in enumerate(cols):
    name = re.sub('form\.','',name)
    name = re.sub('health_centre_information\.','',name)
    name = re.sub('location_information\.','',name)
    name = re.sub('group_number_employed\.','',name)
    name = re.sub('list_.*\.','',name)
    name = re.sub('ql_information_education_communication\.','',name)
    name = re.sub('ql_human_resources\.','',name)
    name = re.sub('ql_surveillance\.','',name)
    name = re.sub('ql_triage_and_early_recognition\.','',name)
    name = re.sub('ql_chw\.','',name)
    name = re.sub('ql_isolation_physical_distancing\.','',name)
    
    # remove everything from 'grp_infection_prevention_and_control.' to the next '.'
    name = re.sub('grp_infection_prevention_and_control\..*\.','',name) 
    # remove any remaining instances of 'grp_infection_prevention_and_control.'
    name = re.sub('grp_infection_prevention_and_control\.','',name) 
    
    # remove everything before the word 'log'
    name = re.sub('.*\.log_','log_',name)
    
    new_cols.append(name)

    print(name)
df.columns = new_cols

facility_name
facility_gps
region_province
district
chiefdom
facility_location
facility_type
facility_type_other
managing_authority
managing_authority_other
setting
outpatient_only
capacity.number_consultation_rooms
capacity.number_inpatient_beds
capacity.number_maternity_beds
cho_number_in_post
cho_number_present_at_visit
cht_number_in_post
cht_number_present_at_visit
cha_number_in_post
cha_number_present_at_visit
cm_number_in_post
cm_number_present_at_visit
sechn_number_in_post
sechn_number_present_at_visit
lab_tech_number_in_post
lab_tech_number_present_at_visit
mch_aides_number_in_post
mch_aides_number_present_at_visit
cleaner_porter_number_in_post
cleaner_porter_number_present_at_visit
other_number_in_post
other_number_present_at_visit
other_staff_cadre
sections_to_review
consultations.head_count.month_1
consultations.head_count.month_2
consultations.head_count.month_3
consultations.head_count.month_4
consultations.general_outpatient.month_1
consultations.general_outpatient.month_

Abbreviations in the new column names:
* cho: community health officer
* cht: community health technician
* cha: community health assistant
* cm: community midwives
* sechn: state enrolled community health nurse
* mch: maternal and child health
* hr: human resources
* iec: information education communication
* surv: surveillance
* ter: triage and early recognition
* chw: community health worker
* iso: isolation
* ppe: personal protective equipment
* ipc: infection prevention and control
* wcd: waste collection and disposal
* ds: disinfection and sterilization
* log: logistics

## Some preliminary changes to the data

There are a few obvious things we can do right off the bat to make the data easier to work with. 

Firstly, the missing values in the dataset are indicated by '---'. Explicitly saying that the values are missing is more useful, so we replace the '---' values with np.nan. 

In [207]:
# replace --- values with NA
df = df.replace('---', np.nan, regex=True)

We can also split the GPS values into latitude and longitude:

In [208]:
# split GPS values into longitude and latitude
df.insert(loc=1,column = 'facility_gps_lat',value=df['facility_gps'].apply(lambda x: x.split(',')[0]))
df.insert(loc=2,column = 'facility_gps_lon',value=df['facility_gps'].apply(lambda x: x.split(',')[1]))
df = df.drop('facility_gps',axis=1)

/tmp/ipykernel_23591/4057685809.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(loc=1,column = 'facility_gps_lat',value=df['facility_gps'].apply(lambda x: x.split(',')[0]))
/tmp/ipykernel_23591/4057685809.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.insert(loc=2,column = 'facility_gps_lon',value=df['facility_gps'].apply(lambda x: x.split(',')[1]))


We can change yes and no values to true and false values:

In [209]:
# replace 'yes' with True and 'no' with False
df['outpatient_only'] = df['outpatient_only'].apply(lambda x: True if x=='yes' else x)
df['outpatient_only'] = df['outpatient_only'].apply(lambda x: False if x=='no' else x)


## Data types

The dataframe will be easier to use for modeling and to work with in general if the data types for the columns are specified. Let us explore the current data types:

In [210]:
df.dtypes

facility_name                                                                 object
facility_gps_lat                                                              object
facility_gps_lon                                                              object
region_province                                                               object
district                                                                      object
chiefdom                                                                      object
facility_location                                                             object
facility_type                                                                 object
facility_type_other                                                          float64
managing_authority                                                            object
managing_authority_other                                                      object
setting                                                          

There are lots of 'object' data types. Let's make the dataframe easier to use for modelling by assigning the correct data types. There are a lot of columns, so I assign data types in a bit of a hacky way below to save time. I consider the part of the column name after the last full stop. If it contains a particular keyword, I give it a particular data type. I use 16 bits for the integers because it's very unlikely that any facility will have more than around 32000 of any of the quantities in this database. 

In [211]:
type_dict = {} # make a dictionary specifying data types for columns 
for col in df.columns:
    name_parts = col.split('.')
    
    if len(re.findall('number_', name_parts[-1]))!=0:
        type_dict[col] = 'Int16'
    elif name_parts[-1][:6]=='month_':
        type_dict[col] = 'Int16'
    elif name_parts[-1][:3]=='hr_':
        type_dict[col] = 'string'
    elif name_parts[-1][:4]=='iec_':
        type_dict[col] = 'string'
    elif name_parts[-1][:5]=='surv_':
        type_dict[col] = 'string'
    elif name_parts[-1][:4]=='ter_':
        type_dict[col] = 'string'
    elif name_parts[-1][:4]=='chw_':
        type_dict[col] = 'string'
    elif name_parts[-1][:4]=='iso_':
        type_dict[col] = 'string'
    elif name_parts[-1][:4]=='ipc_':
        type_dict[col] = 'string'
    elif name_parts[-1][:4]=='log_':
        type_dict[col] = 'string'
    elif name_parts[-1][:15]=='monthly_average':
        type_dict[col] = 'float64'
    elif name_parts[-1][:6]=='score_':
        type_dict[col] = 'float64'
type_dict_2 = {'facility_name': 'string', 'facility_gps_lat': 'float64', 'facility_gps_lon':'float64', 'region_province': 'string', 'district': 'string', 'chiefdom': 'string', 'facility_location': 'string', 'facility_type': 'string', 'facility_type_other': 'string', 'managing_authority': 'string', 'managing_authority_other': 'string', 'setting': 'string', 'outpatient_only': 'boolean' }
type_dict = type_dict | type_dict_2 # join two dictionaries

df=df.astype(type_dict)

We can check that the data types are now correct:

In [212]:
df.dtypes

facility_name                                                                 string
facility_gps_lat                                                             float64
facility_gps_lon                                                             float64
region_province                                                               string
district                                                                      string
chiefdom                                                                      string
facility_location                                                             string
facility_type                                                                 string
facility_type_other                                                           string
managing_authority                                                            string
managing_authority_other                                                      string
setting                                                          

## Specifying case

Python and many other languages are case sensitive, so a word like 'Python' is treated differently from 'python'. We can better detect duplicates if we make the case consistent throughout. 

In [213]:
# get the names of all the string columns
str_cols = [i for i in df.columns if df.dtypes[i]=='string']

# make the string columns all lowercase
df[str_cols] = df[str_cols].apply(lambda x: x.str.lower(),axis=1)

# make location names all uppercase
df['facility_name'] = df['facility_name'].str.upper()
df['region_province'] = df['region_province'].str.upper()
df['district'] = df['district'].str.upper()
df['chiefdom'] = df['chiefdom'].str.upper()
df['facility_location'] = df['facility_location'].str.upper()


## Dealing with duplicates

Now it's time to investigate duplicate data. 

Let's look for duplicate facility names:

In [214]:
df_dup_1 = df[df.duplicated(['facility_name'],keep=False)].sort_values('facility_name')
df_dup_1

,facility_name,facility_gps_lat,facility_gps_lon,region_province,district,chiefdom,facility_location,facility_type,facility_type_other,managing_authority,managing_authority_other,setting,outpatient_only,capacity.number_consultation_rooms,capacity.number_inpatient_beds,capacity.number_maternity_beds,cho_number_in_post,cho_number_present_at_visit,cht_number_in_post,cht_number_present_at_visit,cha_number_in_post,cha_number_present_at_visit,cm_number_in_post,cm_number_present_at_visit,sechn_number_in_post,sechn_number_present_at_visit,lab_tech_number_in_post,lab_tech_number_present_at_visit,mch_aides_number_in_post,mch_aides_number_present_at_visit,cleaner_porter_number_in_post,cleaner_porter_number_present_at_visit,other_number_in_post,other_number_present_at_visit,other_staff_cadre,sections_to_review,consultations.head_count.month_1,consultations.head_count.month_2,consultations.head_count.month_3,consultations.head_count.month_4,consultations.general_outpatient.month_1,consultations.general_outpatient.month_2,consultations.general_outpatient.month_3,consultations.general_outpatient.month_4,consultations.deliveries.month_1,consultations.deliveries.month_2,consultations.deliveries.month_3,consultations.deliveries.month_4,consultations.pent_vaccines.month_1,consultations.pent_vaccines.month_2,consultations.pent_vaccines.month_3,consultations.pent_vaccines.month_4,hr_focal_point,hr_staff_received_info,hr_healthcare_provider_training,hr_healthcare_provider_revised_training,hr_daily_staff_list,iec_handwashing_procedure,iec_physical_distancing,iec_covering_nose_mouth,iec_early_symptom_recognition,iec_when_facility_vs_home,iec_rational_ppe_use,iec_helpline_number,surv_procedure_for_notification,surv_official_case_definition,surv_hotline_number,surv_timely_data_reported_to_district,ter_screening_area_set_up,ter_symptom_screening_questionnaires,ter_temperature_measurement_at_triage,ter_physical_distancing_in_waiting,ter_separate_waiting_for_symptomatic,chw_trained_precautions,chw_trained_community_service,chw_drugs,chw_gloves,chw_masks,chw_iec_materials,iso_designated_isolation_for_suspected,iso_distance_between_patients_in_waiting,iso_distance_between_patient_beds,iso_transfer_referral_protocol,ipc_ppe_medical_masks,ipc_ppe_disp_surgical_masks,ipc_ppe_eye_protection,ipc_ppe_examination_gloves,ipc_ppe_surgical_gloves,ipc_ppe_long_cuffed_gloves,ipc_ppe_heavy_duty_gloves,ipc_ppe_long_sleeved_gown,ipc_ppe_waterproof_aprons,ipc_ppe_plan_staff_trained_on_ppe,ipc_ppe_plan_ppe_poster_displayed,ipc_ppe_plan_fit_test_kit,ipc_ppe_plan_contingency_for_shortages,ipc_wcd_colour_coded_bins,ipc_wcd_clinical_waste_bags,ipc_wcd_laundry_receptacles_at_patient_rooms,ipc_wcd_incinerator,ipc_wash_clean_running_water,ipc_wash_hand_soap,ipc_wash_liquid_soap,ipc_wash_disp_hand_towels,ipc_wash_alcohol_based_hand_gel,ipc_ds_protocol_facility_disinfection,ipc_ds_protocol_equipment_sterilisation,ipc_ds_environmental_disinfectant,ipc_ds_cleaning_schedule_in_toilets,ipc_ds_protocol_corpse_handling,log_referral_plan,log_cellphone_landline_swradio,log_tracer_drugs,log_albendazole,log_amoxicillin,log_ampicillin,log_chlorhexidine_5,log_chlorhexidine_7,log_gentamicin,log_folic,log_ferrous_and_folic,log_compound_sodium,log_co_trimoxazole_400,log_co_trimoxazole_200,log_metronidazole_250,log_metronidazole_200,log_methyldopa,log_magnesium_sulphate,log_lidocaine,log_ibuprofen,log_surgical_spirit,log_sodium_chloride,log_povidone,log_paracetamol_500,log_paracetamol_250,log_ors,log_gauze,log_cotton_wool,log_cannula_iv_20,log_cannula_iv_24,log_zinc_sulphate,log_water,log_needle_23,log_needle_21,log_glove_giving,log_glove_surgical,log_glove_gyn,log_glove_exam,log_tape,log_syringe,log_oxytocin,log_diazepam,log_misoprostol,log_glucose,consultations.head_count.monthly_average_head_counts,consultations.general_outpatient.monthly_average_general_outpatient,consultations.deliveries.monthly_average_deliveries,consultations.pent_vaccines.monthly_average_pent_vaccines,score_human_resources,

Let's see how many elements are in this dataframe:

In [215]:
len(df_dup_1)

40

It is possible that some of the duplicate facility names refer to independent clinics that happen to have the same name. Let's also consider the number of consultations in the facility to get a clearer picture. 

In [216]:
df_dup_2 = df[df.duplicated(['facility_name','consultations.head_count.month_1','consultations.head_count.month_2','consultations.head_count.month_3','consultations.head_count.month_4'],keep=False)].sort_values('facility_name')
df_dup_2

,facility_name,facility_gps_lat,facility_gps_lon,region_province,district,chiefdom,facility_location,facility_type,facility_type_other,managing_authority,managing_authority_other,setting,outpatient_only,capacity.number_consultation_rooms,capacity.number_inpatient_beds,capacity.number_maternity_beds,cho_number_in_post,cho_number_present_at_visit,cht_number_in_post,cht_number_present_at_visit,cha_number_in_post,cha_number_present_at_visit,cm_number_in_post,cm_number_present_at_visit,sechn_number_in_post,sechn_number_present_at_visit,lab_tech_number_in_post,lab_tech_number_present_at_visit,mch_aides_number_in_post,mch_aides_number_present_at_visit,cleaner_porter_number_in_post,cleaner_porter_number_present_at_visit,other_number_in_post,other_number_present_at_visit,other_staff_cadre,sections_to_review,consultations.head_count.month_1,consultations.head_count.month_2,consultations.head_count.month_3,consultations.head_count.month_4,consultations.general_outpatient.month_1,consultations.general_outpatient.month_2,consultations.general_outpatient.month_3,consultations.general_outpatient.month_4,consultations.deliveries.month_1,consultations.deliveries.month_2,consultations.deliveries.month_3,consultations.deliveries.month_4,consultations.pent_vaccines.month_1,consultations.pent_vaccines.month_2,consultations.pent_vaccines.month_3,consultations.pent_vaccines.month_4,hr_focal_point,hr_staff_received_info,hr_healthcare_provider_training,hr_healthcare_provider_revised_training,hr_daily_staff_list,iec_handwashing_procedure,iec_physical_distancing,iec_covering_nose_mouth,iec_early_symptom_recognition,iec_when_facility_vs_home,iec_rational_ppe_use,iec_helpline_number,surv_procedure_for_notification,surv_official_case_definition,surv_hotline_number,surv_timely_data_reported_to_district,ter_screening_area_set_up,ter_symptom_screening_questionnaires,ter_temperature_measurement_at_triage,ter_physical_distancing_in_waiting,ter_separate_waiting_for_symptomatic,chw_trained_precautions,chw_trained_community_service,chw_drugs,chw_gloves,chw_masks,chw_iec_materials,iso_designated_isolation_for_suspected,iso_distance_between_patients_in_waiting,iso_distance_between_patient_beds,iso_transfer_referral_protocol,ipc_ppe_medical_masks,ipc_ppe_disp_surgical_masks,ipc_ppe_eye_protection,ipc_ppe_examination_gloves,ipc_ppe_surgical_gloves,ipc_ppe_long_cuffed_gloves,ipc_ppe_heavy_duty_gloves,ipc_ppe_long_sleeved_gown,ipc_ppe_waterproof_aprons,ipc_ppe_plan_staff_trained_on_ppe,ipc_ppe_plan_ppe_poster_displayed,ipc_ppe_plan_fit_test_kit,ipc_ppe_plan_contingency_for_shortages,ipc_wcd_colour_coded_bins,ipc_wcd_clinical_waste_bags,ipc_wcd_laundry_receptacles_at_patient_rooms,ipc_wcd_incinerator,ipc_wash_clean_running_water,ipc_wash_hand_soap,ipc_wash_liquid_soap,ipc_wash_disp_hand_towels,ipc_wash_alcohol_based_hand_gel,ipc_ds_protocol_facility_disinfection,ipc_ds_protocol_equipment_sterilisation,ipc_ds_environmental_disinfectant,ipc_ds_cleaning_schedule_in_toilets,ipc_ds_protocol_corpse_handling,log_referral_plan,log_cellphone_landline_swradio,log_tracer_drugs,log_albendazole,log_amoxicillin,log_ampicillin,log_chlorhexidine_5,log_chlorhexidine_7,log_gentamicin,log_folic,log_ferrous_and_folic,log_compound_sodium,log_co_trimoxazole_400,log_co_trimoxazole_200,log_metronidazole_250,log_metronidazole_200,log_methyldopa,log_magnesium_sulphate,log_lidocaine,log_ibuprofen,log_surgical_spirit,log_sodium_chloride,log_povidone,log_paracetamol_500,log_paracetamol_250,log_ors,log_gauze,log_cotton_wool,log_cannula_iv_20,log_cannula_iv_24,log_zinc_sulphate,log_water,log_needle_23,log_needle_21,log_glove_giving,log_glove_surgical,log_glove_gyn,log_glove_exam,log_tape,log_syringe,log_oxytocin,log_diazepam,log_misoprostol,log_glucose,consultations.head_count.monthly_average_head_counts,consultations.general_outpatient.monthly_average_general_outpatient,consultations.deliveries.monthly_average_deliveries,consultations.pent_vaccines.monthly_average_pent_vaccines,score_human_resources,

It's very unlikely that the consultation headcount numbers for four months would be exactly the same between two identically named clinics. These are therefore likely to truly refer to the same facility.



In [217]:
len(df_dup_2)

18

Our number of duplicates has now dropped lower. But maybe there are still some very similar entries where one or more of the consultation headcounts differ. We have not recorded those in df_dup_2. Let's search for matches with regard to a number of different column combinations and merge the resulting dataframes into one. 

We consider matching facility names and facility locations first.

In [218]:
df_dup_3 = df[df.duplicated(['facility_name','facility_location'],keep=False)].sort_values('facility_name')
df_dup_3

,facility_name,facility_gps_lat,facility_gps_lon,region_province,district,chiefdom,facility_location,facility_type,facility_type_other,managing_authority,managing_authority_other,setting,outpatient_only,capacity.number_consultation_rooms,capacity.number_inpatient_beds,capacity.number_maternity_beds,cho_number_in_post,cho_number_present_at_visit,cht_number_in_post,cht_number_present_at_visit,cha_number_in_post,cha_number_present_at_visit,cm_number_in_post,cm_number_present_at_visit,sechn_number_in_post,sechn_number_present_at_visit,lab_tech_number_in_post,lab_tech_number_present_at_visit,mch_aides_number_in_post,mch_aides_number_present_at_visit,cleaner_porter_number_in_post,cleaner_porter_number_present_at_visit,other_number_in_post,other_number_present_at_visit,other_staff_cadre,sections_to_review,consultations.head_count.month_1,consultations.head_count.month_2,consultations.head_count.month_3,consultations.head_count.month_4,consultations.general_outpatient.month_1,consultations.general_outpatient.month_2,consultations.general_outpatient.month_3,consultations.general_outpatient.month_4,consultations.deliveries.month_1,consultations.deliveries.month_2,consultations.deliveries.month_3,consultations.deliveries.month_4,consultations.pent_vaccines.month_1,consultations.pent_vaccines.month_2,consultations.pent_vaccines.month_3,consultations.pent_vaccines.month_4,hr_focal_point,hr_staff_received_info,hr_healthcare_provider_training,hr_healthcare_provider_revised_training,hr_daily_staff_list,iec_handwashing_procedure,iec_physical_distancing,iec_covering_nose_mouth,iec_early_symptom_recognition,iec_when_facility_vs_home,iec_rational_ppe_use,iec_helpline_number,surv_procedure_for_notification,surv_official_case_definition,surv_hotline_number,surv_timely_data_reported_to_district,ter_screening_area_set_up,ter_symptom_screening_questionnaires,ter_temperature_measurement_at_triage,ter_physical_distancing_in_waiting,ter_separate_waiting_for_symptomatic,chw_trained_precautions,chw_trained_community_service,chw_drugs,chw_gloves,chw_masks,chw_iec_materials,iso_designated_isolation_for_suspected,iso_distance_between_patients_in_waiting,iso_distance_between_patient_beds,iso_transfer_referral_protocol,ipc_ppe_medical_masks,ipc_ppe_disp_surgical_masks,ipc_ppe_eye_protection,ipc_ppe_examination_gloves,ipc_ppe_surgical_gloves,ipc_ppe_long_cuffed_gloves,ipc_ppe_heavy_duty_gloves,ipc_ppe_long_sleeved_gown,ipc_ppe_waterproof_aprons,ipc_ppe_plan_staff_trained_on_ppe,ipc_ppe_plan_ppe_poster_displayed,ipc_ppe_plan_fit_test_kit,ipc_ppe_plan_contingency_for_shortages,ipc_wcd_colour_coded_bins,ipc_wcd_clinical_waste_bags,ipc_wcd_laundry_receptacles_at_patient_rooms,ipc_wcd_incinerator,ipc_wash_clean_running_water,ipc_wash_hand_soap,ipc_wash_liquid_soap,ipc_wash_disp_hand_towels,ipc_wash_alcohol_based_hand_gel,ipc_ds_protocol_facility_disinfection,ipc_ds_protocol_equipment_sterilisation,ipc_ds_environmental_disinfectant,ipc_ds_cleaning_schedule_in_toilets,ipc_ds_protocol_corpse_handling,log_referral_plan,log_cellphone_landline_swradio,log_tracer_drugs,log_albendazole,log_amoxicillin,log_ampicillin,log_chlorhexidine_5,log_chlorhexidine_7,log_gentamicin,log_folic,log_ferrous_and_folic,log_compound_sodium,log_co_trimoxazole_400,log_co_trimoxazole_200,log_metronidazole_250,log_metronidazole_200,log_methyldopa,log_magnesium_sulphate,log_lidocaine,log_ibuprofen,log_surgical_spirit,log_sodium_chloride,log_povidone,log_paracetamol_500,log_paracetamol_250,log_ors,log_gauze,log_cotton_wool,log_cannula_iv_20,log_cannula_iv_24,log_zinc_sulphate,log_water,log_needle_23,log_needle_21,log_glove_giving,log_glove_surgical,log_glove_gyn,log_glove_exam,log_tape,log_syringe,log_oxytocin,log_diazepam,log_misoprostol,log_glucose,consultations.head_count.monthly_average_head_counts,consultations.general_outpatient.monthly_average_general_outpatient,consultations.deliveries.monthly_average_deliveries,consultations.pent_vaccines.monthly_average_pent_vaccines,score_human_resources,

In [219]:
len(df_dup_3)

24

Now we consider cases where at least one of the months' consultation headcounts match. This seems unlikely enough that it's worth looking into.

In [220]:
# considering rows where consultation headcount month 1 matches
df_dup_4 = df[df.duplicated(['facility_name','consultations.head_count.month_1'],keep=False)].sort_values('facility_name')
# considering rows where consultation headcount month 2 matches
df_dup_5 = df[df.duplicated(['facility_name','consultations.head_count.month_2'],keep=False)].sort_values('facility_name')
# considering rows where consultation headcount month 3 matches
df_dup_6 = df[df.duplicated(['facility_name','consultations.head_count.month_3'],keep=False)].sort_values('facility_name')
# considering rows where consultation headcount month 4 matches
df_dup_7 = df[df.duplicated(['facility_name','consultations.head_count.month_4'],keep=False)].sort_values('facility_name')

There will be a lot of overlap among df_dup_3, df_dup_4, df_dup_5, df_dup_6, and df_dup_7. We want to save all the unique entries across these dataframes, without having extra duplicates. We start by merging the duplicate candidates into one dataframe, in the process creating lots of duplicates. Those duplicates need to be deleted later.  

In [221]:
df_dup_potential_redundant = pd.concat([df_dup_3,df_dup_4,df_dup_5,df_dup_6,df_dup_7])

We now find and delete the duplicate indices. 

In [225]:
df_dup_potential = df_dup_potential_redundant[~df_dup_potential_redundant.index.duplicated(keep='first')]
#dup_indices = df_dup_potential_redundant.index.duplicated(keep='first')
#df_dup_potential = df_dup_potential_redundant.drop(df_dup_potential_redundant[dup_indices].index)
df_dup_potential

,facility_name,facility_gps_lat,facility_gps_lon,region_province,district,chiefdom,facility_location,facility_type,facility_type_other,managing_authority,managing_authority_other,setting,outpatient_only,capacity.number_consultation_rooms,capacity.number_inpatient_beds,capacity.number_maternity_beds,cho_number_in_post,cho_number_present_at_visit,cht_number_in_post,cht_number_present_at_visit,cha_number_in_post,cha_number_present_at_visit,cm_number_in_post,cm_number_present_at_visit,sechn_number_in_post,sechn_number_present_at_visit,lab_tech_number_in_post,lab_tech_number_present_at_visit,mch_aides_number_in_post,mch_aides_number_present_at_visit,cleaner_porter_number_in_post,cleaner_porter_number_present_at_visit,other_number_in_post,other_number_present_at_visit,other_staff_cadre,sections_to_review,consultations.head_count.month_1,consultations.head_count.month_2,consultations.head_count.month_3,consultations.head_count.month_4,consultations.general_outpatient.month_1,consultations.general_outpatient.month_2,consultations.general_outpatient.month_3,consultations.general_outpatient.month_4,consultations.deliveries.month_1,consultations.deliveries.month_2,consultations.deliveries.month_3,consultations.deliveries.month_4,consultations.pent_vaccines.month_1,consultations.pent_vaccines.month_2,consultations.pent_vaccines.month_3,consultations.pent_vaccines.month_4,hr_focal_point,hr_staff_received_info,hr_healthcare_provider_training,hr_healthcare_provider_revised_training,hr_daily_staff_list,iec_handwashing_procedure,iec_physical_distancing,iec_covering_nose_mouth,iec_early_symptom_recognition,iec_when_facility_vs_home,iec_rational_ppe_use,iec_helpline_number,surv_procedure_for_notification,surv_official_case_definition,surv_hotline_number,surv_timely_data_reported_to_district,ter_screening_area_set_up,ter_symptom_screening_questionnaires,ter_temperature_measurement_at_triage,ter_physical_distancing_in_waiting,ter_separate_waiting_for_symptomatic,chw_trained_precautions,chw_trained_community_service,chw_drugs,chw_gloves,chw_masks,chw_iec_materials,iso_designated_isolation_for_suspected,iso_distance_between_patients_in_waiting,iso_distance_between_patient_beds,iso_transfer_referral_protocol,ipc_ppe_medical_masks,ipc_ppe_disp_surgical_masks,ipc_ppe_eye_protection,ipc_ppe_examination_gloves,ipc_ppe_surgical_gloves,ipc_ppe_long_cuffed_gloves,ipc_ppe_heavy_duty_gloves,ipc_ppe_long_sleeved_gown,ipc_ppe_waterproof_aprons,ipc_ppe_plan_staff_trained_on_ppe,ipc_ppe_plan_ppe_poster_displayed,ipc_ppe_plan_fit_test_kit,ipc_ppe_plan_contingency_for_shortages,ipc_wcd_colour_coded_bins,ipc_wcd_clinical_waste_bags,ipc_wcd_laundry_receptacles_at_patient_rooms,ipc_wcd_incinerator,ipc_wash_clean_running_water,ipc_wash_hand_soap,ipc_wash_liquid_soap,ipc_wash_disp_hand_towels,ipc_wash_alcohol_based_hand_gel,ipc_ds_protocol_facility_disinfection,ipc_ds_protocol_equipment_sterilisation,ipc_ds_environmental_disinfectant,ipc_ds_cleaning_schedule_in_toilets,ipc_ds_protocol_corpse_handling,log_referral_plan,log_cellphone_landline_swradio,log_tracer_drugs,log_albendazole,log_amoxicillin,log_ampicillin,log_chlorhexidine_5,log_chlorhexidine_7,log_gentamicin,log_folic,log_ferrous_and_folic,log_compound_sodium,log_co_trimoxazole_400,log_co_trimoxazole_200,log_metronidazole_250,log_metronidazole_200,log_methyldopa,log_magnesium_sulphate,log_lidocaine,log_ibuprofen,log_surgical_spirit,log_sodium_chloride,log_povidone,log_paracetamol_500,log_paracetamol_250,log_ors,log_gauze,log_cotton_wool,log_cannula_iv_20,log_cannula_iv_24,log_zinc_sulphate,log_water,log_needle_23,log_needle_21,log_glove_giving,log_glove_surgical,log_glove_gyn,log_glove_exam,log_tape,log_syringe,log_oxytocin,log_diazepam,log_misoprostol,log_glucose,consultations.head_count.monthly_average_head_counts,consultations.general_outpatient.monthly_average_general_outpatient,consultations.deliveries.monthly_average_deliveries,consultations.pent_vaccines.monthly_average_pent_vaccines,score_human_resources,

I'm confident that the duplicates in df_dup_2 are genuine duplicates, so I'm going to remove them from df_dup_potential to make it easier to examine the more dubious cases. 

In [226]:
dup_2_indices = list(df_dup_2.index)

In [227]:
df_dup_potential_2 = df_dup_potential.drop(dup_2_indices,axis=0)

In [228]:
df_dup_potential_2

,facility_name,facility_gps_lat,facility_gps_lon,region_province,district,chiefdom,facility_location,facility_type,facility_type_other,managing_authority,managing_authority_other,setting,outpatient_only,capacity.number_consultation_rooms,capacity.number_inpatient_beds,capacity.number_maternity_beds,cho_number_in_post,cho_number_present_at_visit,cht_number_in_post,cht_number_present_at_visit,cha_number_in_post,cha_number_present_at_visit,cm_number_in_post,cm_number_present_at_visit,sechn_number_in_post,sechn_number_present_at_visit,lab_tech_number_in_post,lab_tech_number_present_at_visit,mch_aides_number_in_post,mch_aides_number_present_at_visit,cleaner_porter_number_in_post,cleaner_porter_number_present_at_visit,other_number_in_post,other_number_present_at_visit,other_staff_cadre,sections_to_review,consultations.head_count.month_1,consultations.head_count.month_2,consultations.head_count.month_3,consultations.head_count.month_4,consultations.general_outpatient.month_1,consultations.general_outpatient.month_2,consultations.general_outpatient.month_3,consultations.general_outpatient.month_4,consultations.deliveries.month_1,consultations.deliveries.month_2,consultations.deliveries.month_3,consultations.deliveries.month_4,consultations.pent_vaccines.month_1,consultations.pent_vaccines.month_2,consultations.pent_vaccines.month_3,consultations.pent_vaccines.month_4,hr_focal_point,hr_staff_received_info,hr_healthcare_provider_training,hr_healthcare_provider_revised_training,hr_daily_staff_list,iec_handwashing_procedure,iec_physical_distancing,iec_covering_nose_mouth,iec_early_symptom_recognition,iec_when_facility_vs_home,iec_rational_ppe_use,iec_helpline_number,surv_procedure_for_notification,surv_official_case_definition,surv_hotline_number,surv_timely_data_reported_to_district,ter_screening_area_set_up,ter_symptom_screening_questionnaires,ter_temperature_measurement_at_triage,ter_physical_distancing_in_waiting,ter_separate_waiting_for_symptomatic,chw_trained_precautions,chw_trained_community_service,chw_drugs,chw_gloves,chw_masks,chw_iec_materials,iso_designated_isolation_for_suspected,iso_distance_between_patients_in_waiting,iso_distance_between_patient_beds,iso_transfer_referral_protocol,ipc_ppe_medical_masks,ipc_ppe_disp_surgical_masks,ipc_ppe_eye_protection,ipc_ppe_examination_gloves,ipc_ppe_surgical_gloves,ipc_ppe_long_cuffed_gloves,ipc_ppe_heavy_duty_gloves,ipc_ppe_long_sleeved_gown,ipc_ppe_waterproof_aprons,ipc_ppe_plan_staff_trained_on_ppe,ipc_ppe_plan_ppe_poster_displayed,ipc_ppe_plan_fit_test_kit,ipc_ppe_plan_contingency_for_shortages,ipc_wcd_colour_coded_bins,ipc_wcd_clinical_waste_bags,ipc_wcd_laundry_receptacles_at_patient_rooms,ipc_wcd_incinerator,ipc_wash_clean_running_water,ipc_wash_hand_soap,ipc_wash_liquid_soap,ipc_wash_disp_hand_towels,ipc_wash_alcohol_based_hand_gel,ipc_ds_protocol_facility_disinfection,ipc_ds_protocol_equipment_sterilisation,ipc_ds_environmental_disinfectant,ipc_ds_cleaning_schedule_in_toilets,ipc_ds_protocol_corpse_handling,log_referral_plan,log_cellphone_landline_swradio,log_tracer_drugs,log_albendazole,log_amoxicillin,log_ampicillin,log_chlorhexidine_5,log_chlorhexidine_7,log_gentamicin,log_folic,log_ferrous_and_folic,log_compound_sodium,log_co_trimoxazole_400,log_co_trimoxazole_200,log_metronidazole_250,log_metronidazole_200,log_methyldopa,log_magnesium_sulphate,log_lidocaine,log_ibuprofen,log_surgical_spirit,log_sodium_chloride,log_povidone,log_paracetamol_500,log_paracetamol_250,log_ors,log_gauze,log_cotton_wool,log_cannula_iv_20,log_cannula_iv_24,log_zinc_sulphate,log_water,log_needle_23,log_needle_21,log_glove_giving,log_glove_surgical,log_glove_gyn,log_glove_exam,log_tape,log_syringe,log_oxytocin,log_diazepam,log_misoprostol,log_glucose,consultations.head_count.monthly_average_head_counts,consultations.general_outpatient.monthly_average_general_outpatient,consultations.deliveries.monthly_average_deliveries,consultations.pent_vaccines.monthly_average_pent_vaccines,score_human_resources,

Now we're left with only four more potential duplicates to investigate.

For each potential match, let's make a mini dataframe.

In [250]:
display(df_dup_potential_2['facility_name'].isnull())

number
26     False
95     False
133    False
39     False
97     False
96     False
5       True
119     True
Name: facility_name, dtype: bool

In [272]:
mini_dfs = []
mini_df_dict = {}
clinic_names = df_dup_potential_2['facility_name'].unique()
for name in clinic_names:
    if not pd.isnull(name): # if the facility name is not null
        mini_df = df_dup_potential_2[df_dup_potential_2['facility_name']==name]
    else: # when the facility name is null   
        mini_df = df_dup_potential_2[df_dup_potential_2['facility_name'].isnull()]
    mini_df_dict[name]=mini_df
    mini_dfs.append(mini_df)

In [271]:
for i,mini_df in enumerate(mini_dfs):
    facility_name = mini_df['facility_name'].iloc[0]
    print(f'Facility name: {facility_name}')
    indices = list(mini_df.index)
    print('Where the rows differ: ')
    #display(~(pd.isnull(mini_df.loc[indices[0]]) & pd.isnull(mini_df.loc[indices[1]])))
    display(mini_df[mini_df.columns[(mini_df.loc[indices[0]] != mini_df.loc[indices[1]]) & (~(pd.isnull(mini_df.loc[indices[0]]) & pd.isnull(mini_df.loc[indices[1]])))]])
    print('Where the rows are the same: ')
    display(mini_df[mini_df.columns[(mini_df.loc[indices[0]] == mini_df.loc[indices[1]]) | (pd.isnull(mini_df.loc[indices[0]]) & pd.isnull(mini_df.loc[indices[1]]))]])

Facility name: JOHN CLINIC
Where the rows differ: 


,facility_gps_lat,facility_gps_lon,capacity.number_consultation_rooms,capacity.number_inpatient_beds,capacity.number_maternity_beds,cho_number_in_post,cho_number_present_at_visit,cht_number_in_post,cht_number_present_at_visit,cm_number_in_post,cm_number_present_at_visit,sechn_number_in_post,sechn_number_present_at_visit,mch_aides_number_in_post,mch_aides_number_present_at_visit,cleaner_porter_number_in_post,cleaner_porter_number_present_at_visit,other_number_in_post,other_number_present_at_visit,other_staff_cadre,sections_to_review,consultations.head_count.month_1,consultations.head_count.month_2,consultations.head_count.month_3,consultations.head_count.month_4,consultations.general_outpatient.month_1,consultations.general_outpatient.month_2,consultations.general_outpatient.month_3,consultations.general_outpatient.month_4,consultations.deliveries.month_1,consultations.deliveries.month_2,consultations.deliveries.month_3,consultations.deliveries.month_4,consultations.pent_vaccines.month_1,consultations.pent_vaccines.month_2,consultations.pent_vaccines.month_3,consultations.pent_vaccines.month_4,hr_focal_point,hr_staff_received_info,hr_healthcare_provider_training,hr_healthcare_provider_revised_training,hr_daily_staff_list,iec_handwashing_procedure,iec_physical_distancing,iec_covering_nose_mouth,iec_early_symptom_recognition,iec_when_facility_vs_home,iec_rational_ppe_use,iec_helpline_number,surv_procedure_for_notification,surv_official_case_definition,surv_hotline_number,surv_timely_data_reported_to_district,ter_screening_area_set_up,ter_symptom_screening_questionnaires,ter_temperature_measurement_at_triage,ter_physical_distancing_in_waiting,ter_separate_waiting_for_symptomatic,chw_trained_precautions,chw_trained_community_service,chw_drugs,chw_gloves,chw_masks,chw_iec_materials,iso_designated_isolation_for_suspected,iso_distance_between_patients_in_waiting,iso_distance_between_patient_beds,iso_transfer_referral_protocol,consultations.head_count.monthly_average_head_counts,consultations.general_outpatient.monthly_average_general_outpatient,consultations.deliveries.monthly_average_deliveries,consultations.pent_vaccines.monthly_average_pent_vaccines,score_human_resources,score_max_human_resources,score_information_education_communication,score_max_information_education_communication,score_surveillance,score_max_surveillance,score_triage_and_early_recognition,score_max_triage_and_early_recognition,score_chw,score_max_chw,score_isolation,score_max_isolation,score_total,score_max_total
number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
26,-54.388258,-131.023446,0,0,1,2,1,0,0,0,0,0,0,2,1,1,1,3,3,TBA,consultations hr information_education_communi...,340,327,368,364,272,267,272,233,4,5,9,6,8,9,9,8,not_completed,completed,not_completed,not_completed,completed,displayed,displayed,displayed,displayed,displayed,displayed,displayed,fully_operational,fully_operational,fully_operational,fully_operational,not_in_place,not_in_place,not_in_place,not_in_place,not_in_place,adequate,adequate,adequate,adequate,adequate,adequate,not_in_place,fully_operational,fully_operational,fully_operational,349.75,261.0,6.00,8.5,2.0,5.0,7.0,7.0,4.0,4.0,0.0,5.0,5.0,6.0,3.0,4.0,21.0,31.0
95,53.647814,-115.559792,1,2,2,15,12,4,2,3,2,5,3,5,4,3,2,0,0,NaN,consultations,14,25,10,11,12,8,10,10,5,4,5,3,23,22,10,15,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,15.00,10.0,4.25,17.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


Where the rows are the same: 


,facility_name,region_province,district,chiefdom,facility_location,facility_type,facility_type_other,managing_authority,managing_authority_other,setting,outpatient_only,cha_number_in_post,cha_number_present_at_visit,lab_tech_number_in_post,lab_tech_number_present_at_visit,ipc_ppe_medical_masks,ipc_ppe_disp_surgical_masks,ipc_ppe_eye_protection,ipc_ppe_examination_gloves,ipc_ppe_surgical_gloves,ipc_ppe_long_cuffed_gloves,ipc_ppe_heavy_duty_gloves,ipc_ppe_long_sleeved_gown,ipc_ppe_waterproof_aprons,ipc_ppe_plan_staff_trained_on_ppe,ipc_ppe_plan_ppe_poster_displayed,ipc_ppe_plan_fit_test_kit,ipc_ppe_plan_contingency_for_shortages,ipc_wcd_colour_coded_bins,ipc_wcd_clinical_waste_bags,ipc_wcd_laundry_receptacles_at_patient_rooms,ipc_wcd_incinerator,ipc_wash_clean_running_water,ipc_wash_hand_soap,ipc_wash_liquid_soap,ipc_wash_disp_hand_towels,ipc_wash_alcohol_based_hand_gel,ipc_ds_protocol_facility_disinfection,ipc_ds_protocol_equipment_sterilisation,ipc_ds_environmental_disinfectant,ipc_ds_cleaning_schedule_in_toilets,ipc_ds_protocol_corpse_handling,log_referral_plan,log_cellphone_landline_swradio,log_tracer_drugs,log_albendazole,log_amoxicillin,log_ampicillin,log_chlorhexidine_5,log_chlorhexidine_7,log_gentamicin,log_folic,log_ferrous_and_folic,log_compound_sodium,log_co_trimoxazole_400,log_co_trimoxazole_200,log_metronidazole_250,log_metronidazole_200,log_methyldopa,log_magnesium_sulphate,log_lidocaine,log_ibuprofen,log_surgical_spirit,log_sodium_chloride,log_povidone,log_paracetamol_500,log_paracetamol_250,log_ors,log_gauze,log_cotton_wool,log_cannula_iv_20,log_cannula_iv_24,log_zinc_sulphate,log_water,log_needle_23,log_needle_21,log_glove_giving,log_glove_surgical,log_glove_gyn,log_glove_exam,log_tape,log_syringe,log_oxytocin,log_diazepam,log_misoprostol,log_glucose,score_infection_prevention_and_control_ppe,score_max_infection_prevention_and_control_ppe,score_infection_prevention_and_control_ppe_plan,score_max_infection_prevention_and_control_ppe_plan,score_infection_prevention_and_control_waste_collection_and_disposal,score_max_infection_prevention_and_control_waste_collection_and_disposal,score_infection_prevention_and_control_water_sanitation_and_hygiene,score_max_infection_prevention_and_control_water_sanitation_and_hygiene,score_infection_prevention_and_control_disinfection_and_sterilization,score_max_infection_prevention_and_control_disinfection_and_sterilization,score_infection_prevention_and_control,score_max_infection_prevention_and_control,question1.score_logistics_patient_and_sample_transfer,question1.score_max_logistics_patient_and_sample_transfer
number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
26,JOHN CLINIC,NORTHERN,KOINADUGU,TENNESSEE,GARYTOWN,mch_post,<NA>,government_public,<NA>,peri_urban,True,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,JOHN CLINIC,NORTHERN,KOINADUGU,TENNESSEE,GARYTOWN,mch_post,<NA>,government_public,<NA>,peri_urban,True,0,0,0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Facility name: KEVIN HOSPITAL
Where the rows differ: 


...

,facility_name,region_province,district,chiefdom,facility_location,facility_type,facility_type_other,managing_authority,managing_authority_other,setting,outpatient_only,capacity.number_consultation_rooms,capacity.number_inpatient_beds,capacity.number_maternity_beds,cho_number_in_post,cho_number_present_at_visit,cht_number_in_post,cht_number_present_at_visit,cha_number_in_post,cha_number_present_at_visit,cm_number_in_post,sechn_number_in_post,sechn_number_present_at_visit,lab_tech_number_in_post,lab_tech_number_present_at_visit,mch_aides_number_in_post,mch_aides_number_present_at_visit,cleaner_porter_number_in_post,cleaner_porter_number_present_at_visit,other_number_in_post,other_number_present_at_visit,other_staff_cadre,consultations.head_count.month_2,consultations.head_count.month_3,consultations.head_count.month_4,consultations.general_outpatient.month_2,consultations.general_outpatient.month_3,consultations.general_outpatient.month_4,consultations.deliveries.month_1,consultations.deliveries.month_2,consultations.deliveries.month_3,consultations.deliveries.month_4,consultations.pent_vaccines.month_1,consultations.pent_vaccines.month_2,consultations.pent_vaccines.month_3,consultations.pent_vaccines.month_4,ipc_ppe_examination_gloves,consultations.deliveries.monthly_average_deliveries,consultations.pent_vaccines.monthly_average_pent_vaccines
number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
97,RICHARD MCHP,WESTERN,WESTERN AREA URBAN,KENTUCKY,COLLEENBURGH,community_health_post,<NA>,other,private for fhc,urban,True,1,6,2,0,0,0,0,0,0,1,1,0,0,0,4,0,0,0,1,0,nursing aide,1266,1079,887,883,811,607,20,19,23,16,38,42,21,19,<NA>,19.5,30.0
96,RICHARD MCHP,WESTERN,WESTERN AREA URBAN,KENTUCKY,COLLEENBURGH,community_health_post,<NA>,other,private for fhc,urban,True,1,6,2,0,0,0,0,0,0,1,1,0,0,0,4,0,0,0,1,0,nursing aide,1266,1079,887,883,811,607,20,19,23,16,38,42,21,19,<NA>,19.5,30.0


Facility name: <NA>
Where the rows differ: 


,facility_gps_lat,facility_gps_lon,consultations.head_count.month_1,consultations.head_count.month_2,consultations.head_count.month_3,consultations.head_count.month_4,consultations.general_outpatient.month_1,consultations.general_outpatient.month_2,consultations.general_outpatient.month_3,consultations.general_outpatient.month_4,consultations.deliveries.month_1,consultations.deliveries.month_2,consultations.deliveries.month_3,consultations.deliveries.month_4,consultations.pent_vaccines.month_1,consultations.pent_vaccines.month_2,consultations.pent_vaccines.month_3,consultations.pent_vaccines.month_4,hr_focal_point,hr_staff_received_info,iec_handwashing_procedure,iec_early_symptom_recognition,iec_when_facility_vs_home,surv_hotline_number,surv_timely_data_reported_to_district,chw_trained_precautions,chw_trained_community_service,chw_drugs,chw_gloves,chw_masks,chw_iec_materials,ipc_ppe_plan_staff_trained_on_ppe,ipc_ppe_plan_ppe_poster_displayed,ipc_ppe_plan_fit_test_kit,ipc_ppe_plan_contingency_for_shortages,ipc_wash_clean_running_water,ipc_wash_hand_soap,ipc_wash_liquid_soap,ipc_wash_disp_hand_towels,ipc_wash_alcohol_based_hand_gel,ipc_ds_protocol_facility_disinfection,ipc_ds_protocol_equipment_sterilisation,ipc_ds_environmental_disinfectant,ipc_ds_cleaning_schedule_in_toilets,log_referral_plan,log_cellphone_landline_swradio,log_tracer_drugs,log_albendazole,log_chlorhexidine_5,consultations.head_count.monthly_average_head_counts,consultations.general_outpatient.monthly_average_general_outpatient,consultations.deliveries.monthly_average_deliveries,consultations.pent_vaccines.monthly_average_pent_vaccines,score_human_resources,score_information_education_communication,score_chw,score_infection_prevention_and_control_ppe_plan,score_infection_prevention_and_control_water_sanitation_and_hygiene,score_infection_prevention_and_control_disinfection_and_sterilization,score_infection_prevention_and_control
number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,22.856753,19.262410,23,23,23,45,85,85,84,86,36,35,12,58,120,125,639,588,partially_completed,not_completed,not_displayed,displayed,not_displayed,partially_operational,fully_operational,partial,partial,adequate,partial,partial,adequate,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,28.5,85.00,35.25,368.00,2.0,3.0,3.5,0.0,0.0,0.0,0.0
119,-1.129569,-34.179477,56,5,53,34,23,3,45,656,12,1,415,12,45,4,45,45,completed,partially_completed,displayed,not_displayed,displayed,fully_operational,partially_operational,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,partially_achieved,available_fully_achieved,not_available,partially_achieved,sufficient_supplies,risk_of_shortage,sufficient_supplies,risk_of_shortage,not_available,risk_of_shortage,sufficient_supplies,not_available,risk_of_shortage,available,partially_available,available,partially_available,partially_available,37.0,181.75,110.00,34.75,3.0,4.0,0.0,2.0,3.0,1.5,6.5


Where the rows are the same: 


,facility_name,region_province,district,chiefdom,facility_location,facility_type,facility_type_other,managing_authority,managing_authority_other,setting,outpatient_only,capacity.number_consultation_rooms,capacity.number_inpatient_beds,capacity.number_maternity_beds,cho_number_in_post,cho_number_present_at_visit,cht_number_in_post,cht_number_present_at_visit,cha_number_in_post,cha_number_present_at_visit,cm_number_in_post,cm_number_present_at_visit,sechn_number_in_post,sechn_number_present_at_visit,lab_tech_number_in_post,lab_tech_number_present_at_visit,mch_aides_number_in_post,mch_aides_number_present_at_visit,cleaner_porter_number_in_post,cleaner_porter_number_present_at_visit,other_number_in_post,other_number_present_at_visit,other_staff_cadre,sections_to_review,hr_healthcare_provider_training,hr_healthcare_provider_revised_training,hr_daily_staff_list,iec_physical_distancing,iec_covering_nose_mouth,iec_rational_ppe_use,iec_helpline_number,surv_procedure_for_notification,surv_official_case_definition,ter_screening_area_set_up,ter_symptom_screening_questionnaires,ter_temperature_measurement_at_triage,ter_physical_distancing_in_waiting,ter_separate_waiting_for_symptomatic,iso_designated_isolation_for_suspected,iso_distance_between_patients_in_waiting,iso_distance_between_patient_beds,iso_transfer_referral_protocol,ipc_ppe_medical_masks,ipc_ppe_disp_surgical_masks,ipc_ppe_eye_protection,ipc_ppe_examination_gloves,ipc_ppe_surgical_gloves,ipc_ppe_long_cuffed_gloves,ipc_ppe_heavy_duty_gloves,ipc_ppe_long_sleeved_gown,ipc_ppe_waterproof_aprons,ipc_wcd_colour_coded_bins,ipc_wcd_clinical_waste_bags,ipc_wcd_laundry_receptacles_at_patient_rooms,ipc_wcd_incinerator,ipc_ds_protocol_corpse_handling,log_amoxicillin,log_ampicillin,log_chlorhexidine_7,log_gentamicin,log_folic,log_ferrous_and_folic,log_compound_sodium,log_co_trimoxazole_400,log_co_trimoxazole_200,log_metronidazole_250,log_metronidazole_200,log_methyldopa,log_magnesium_sulphate,log_lidocaine,log_ibuprofen,log_surgical_spirit,log_sodium_chloride,log_povidone,log_paracetamol_500,log_paracetamol_250,log_ors,log_gauze,log_cotton_wool,log_cannula_iv_20,log_cannula_iv_24,log_zinc_sulphate,log_water,log_needle_23,log_needle_21,log_glove_giving,log_glove_surgical,log_glove_gyn,log_glove_exam,log_tape,log_syringe,log_oxytocin,log_diazepam,log_misoprostol,log_glucose,score_max_human_resources,score_max_information_education_communication,score_surveillance,score_max_surveillance,score_triage_and_early_recognition,score_max_triage_and_early_recognition,score_max_chw,score_isolation,score_max_isolation,score_infection_prevention_and_control_ppe,score_max_infection_prevention_and_control_ppe,score_max_infection_prevention_and_control_ppe_plan,score_infection_prevention_and_control_waste_collection_and_disposal,score_max_infection_prevention_and_control_waste_collection_and_disposal,score_max_infection_prevention_and_control_water_sanitation_and_hygiene,score_max_infection_prevention_and_control_disinfection_and_sterilization,score_max_infection_prevention_and_control,question1.score_logistics_patient_and_sample_transfer,question1.score_max_logistics_patient_and_sample_transfer,score_total,score_max_total
number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,consultations hr information_education_communi...,completed,partially_completed,not_completed,not_displayed,displayed,displayed,not_displayed,fully_operational,partially_operational,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,not_available,partially_available,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>

* JOHN CLINIC: The two rows are sufficiently different that I'm not sure enough that this is the same clinic to merge the rows into one or delete one of them. I therefore leave them as is. 
* KEVIN HOSPITAL: The two rows match in enough places that they are probably referring to the same clinic. They still differ in many places, however. I will merge them, but record both originals in a separate file and make the merging easy to comment out.  
* RICHARD MCHP: The two rows match in enough places that they are probably referring to the same clinic. They still differ in many places, however. I will merge them, but record both originals in a separate file and make the merging easy to comment out.  
* N/A: The rows are very different, so I leave these as two facilities. 


### Merging rows

I will merge the following rows:
* KEVIN HOSPITAL (indices 133 and 39)
* RICHARD MCHP (indices 96 and 97)
* All the facility name pairs in df_dup_2

Before doing any merging, I will export the duplicates to a csv file so that the machine learning engineer can examine it. 

In [283]:
df_dup = pd.concat([df_dup_2,mini_df_dict['KEVIN HOSPITAL'],mini_df_dict['RICHARD MCHP']])
print(f'Duplicates in df_dup: {sum(df_dup.index.duplicated())}')
df_dup.to_csv('similar_rows.csv')
df_dup

Duplicates in df_dup: 0


,facility_name,facility_gps_lat,facility_gps_lon,region_province,district,chiefdom,facility_location,facility_type,facility_type_other,managing_authority,managing_authority_other,setting,outpatient_only,capacity.number_consultation_rooms,capacity.number_inpatient_beds,capacity.number_maternity_beds,cho_number_in_post,cho_number_present_at_visit,cht_number_in_post,cht_number_present_at_visit,cha_number_in_post,cha_number_present_at_visit,cm_number_in_post,cm_number_present_at_visit,sechn_number_in_post,sechn_number_present_at_visit,lab_tech_number_in_post,lab_tech_number_present_at_visit,mch_aides_number_in_post,mch_aides_number_present_at_visit,cleaner_porter_number_in_post,cleaner_porter_number_present_at_visit,other_number_in_post,other_number_present_at_visit,other_staff_cadre,sections_to_review,consultations.head_count.month_1,consultations.head_count.month_2,consultations.head_count.month_3,consultations.head_count.month_4,consultations.general_outpatient.month_1,consultations.general_outpatient.month_2,consultations.general_outpatient.month_3,consultations.general_outpatient.month_4,consultations.deliveries.month_1,consultations.deliveries.month_2,consultations.deliveries.month_3,consultations.deliveries.month_4,consultations.pent_vaccines.month_1,consultations.pent_vaccines.month_2,consultations.pent_vaccines.month_3,consultations.pent_vaccines.month_4,hr_focal_point,hr_staff_received_info,hr_healthcare_provider_training,hr_healthcare_provider_revised_training,hr_daily_staff_list,iec_handwashing_procedure,iec_physical_distancing,iec_covering_nose_mouth,iec_early_symptom_recognition,iec_when_facility_vs_home,iec_rational_ppe_use,iec_helpline_number,surv_procedure_for_notification,surv_official_case_definition,surv_hotline_number,surv_timely_data_reported_to_district,ter_screening_area_set_up,ter_symptom_screening_questionnaires,ter_temperature_measurement_at_triage,ter_physical_distancing_in_waiting,ter_separate_waiting_for_symptomatic,chw_trained_precautions,chw_trained_community_service,chw_drugs,chw_gloves,chw_masks,chw_iec_materials,iso_designated_isolation_for_suspected,iso_distance_between_patients_in_waiting,iso_distance_between_patient_beds,iso_transfer_referral_protocol,ipc_ppe_medical_masks,ipc_ppe_disp_surgical_masks,ipc_ppe_eye_protection,ipc_ppe_examination_gloves,ipc_ppe_surgical_gloves,ipc_ppe_long_cuffed_gloves,ipc_ppe_heavy_duty_gloves,ipc_ppe_long_sleeved_gown,ipc_ppe_waterproof_aprons,ipc_ppe_plan_staff_trained_on_ppe,ipc_ppe_plan_ppe_poster_displayed,ipc_ppe_plan_fit_test_kit,ipc_ppe_plan_contingency_for_shortages,ipc_wcd_colour_coded_bins,ipc_wcd_clinical_waste_bags,ipc_wcd_laundry_receptacles_at_patient_rooms,ipc_wcd_incinerator,ipc_wash_clean_running_water,ipc_wash_hand_soap,ipc_wash_liquid_soap,ipc_wash_disp_hand_towels,ipc_wash_alcohol_based_hand_gel,ipc_ds_protocol_facility_disinfection,ipc_ds_protocol_equipment_sterilisation,ipc_ds_environmental_disinfectant,ipc_ds_cleaning_schedule_in_toilets,ipc_ds_protocol_corpse_handling,log_referral_plan,log_cellphone_landline_swradio,log_tracer_drugs,log_albendazole,log_amoxicillin,log_ampicillin,log_chlorhexidine_5,log_chlorhexidine_7,log_gentamicin,log_folic,log_ferrous_and_folic,log_compound_sodium,log_co_trimoxazole_400,log_co_trimoxazole_200,log_metronidazole_250,log_metronidazole_200,log_methyldopa,log_magnesium_sulphate,log_lidocaine,log_ibuprofen,log_surgical_spirit,log_sodium_chloride,log_povidone,log_paracetamol_500,log_paracetamol_250,log_ors,log_gauze,log_cotton_wool,log_cannula_iv_20,log_cannula_iv_24,log_zinc_sulphate,log_water,log_needle_23,log_needle_21,log_glove_giving,log_glove_surgical,log_glove_gyn,log_glove_exam,log_tape,log_syringe,log_oxytocin,log_diazepam,log_misoprostol,log_glucose,consultations.head_count.monthly_average_head_counts,consultations.general_outpatient.monthly_average_general_outpatient,consultations.deliveries.monthly_average_deliveries,consultations.pent_vaccines.monthly_average_pent_vaccines,score_human_resources,

In [ ]:
df_amy_1 = df.loc[48]
df_amy_2 = df.loc[87]
df_amy_1

In [ ]:
df_amy_combo = df_amy_1.combine_first(df_amy_2)
df_amy_combo

In [ ]:
df = pd.concat([df, df_amy_combo.to_frame().T], ignore_index=True)
df = df.drop([48,87],axis=0)
df[df['facility_name']=='AMY CHP']

A number of facilities have the same name but are in different states. Since it is fairly unlikely that the state would be recorded incorrectly, these are likely cases where two independent facilities in different states happen to have the same name. Let's therefore require that the facility name and state are both the same and see if this reduces the number of duplicates. 

In [ ]:
#df[df.duplicated(['facility_name','facility_location'],keep=False)].sort_values('facility_location').to_csv('duplicates.csv')

In [ ]:
df[df.duplicated(['facility_name','chiefdom'],keep=False)].sort_values('facility_name')

It is possible that some duplicates are 'hiding' because the names are spelled slightly differently. Let's see if we can catch any like that by printing out all the facilities that have location matches with other facilities. 

In [ ]:
df[df.duplicated(['facility_location'],keep=False)].sort_values('facility_location')

From the above, it looks like there are no cases where facilities in the same location have two slightly different names. 

In [ ]:
print(df['form.health_centre_information.location_information.facility_location'].unique())

In [ ]:
df.to_csv('health_facility_assessment_cleaned.csv', header=True, index=True, index_label=None)